
Проверим утверждение про затухание градиента на практике. В документации pytorch можно найти следующие функции активации: 

ELU, Hardtanh, LeakyReLU, LogSigmoid, PReLU, ReLU, ReLU6, RReLU, SELU, CELU, Sigmoid, Softplus, Softshrink, Softsign, Tanh, Tanhshrink, Hardshrink.

Вам предстоит найти активацию, которая приводит к наименьшему затуханию градиента. 

Для проверки мы сконструируем SimpleNet, которая будет иметь внутри 3 fc-слоя, по 1 нейрону в каждом без bias'ов. Веса этих нейронов мы проинициализируем единицами. На вход в эту сеть будем подавать числа из нормального распределения. Сделаем 200 запусков (NUMBER_OF_EXPERIMENTS) для честного сравнения и посчитаем среднее значение градиента в первом слое. Найдите такую функцию, которая будет давать максимальные значения градиента в первом слое. Все функции активации нужно инициализировать с аргументами по умолчанию (пустыми скобками).

In [3]:
import torch
import numpy as np


np.random.seed(11)
torch.manual_seed(11)

NUMBER_OF_EXPERIMENTS = 200

class SimpleNet(torch.nn.Module):
    def __init__(self, activation):
        super().__init__()

        self.activation = activation
        self.fc1 = torch.nn.Linear(1, 1, bias=False)  # one neuron without bias
        self.fc1.weight.data.fill_(1.)  # init weight with 1
        self.fc2 = torch.nn.Linear(1, 1, bias=False)
        self.fc2.weight.data.fill_(1.)
        self.fc3 = torch.nn.Linear(1, 1, bias=False)
        self.fc3.weight.data.fill_(1.)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        return x

    def get_fc1_grad_abs_value(self):
        return torch.abs(self.fc1.weight.grad)

def get_fc1_grad_abs_value(net, x):
    output = net.forward(x)
    output.backward()  # no loss function. Pretending that we want to minimize output
                       # In our case output is scalar, so we can calculate backward
    fc1_grad = net.get_fc1_grad_abs_value().item()
    net.zero_grad()
    return fc1_grad

In [4]:
activations =  {'ELU': torch.nn.ELU(), 'Hardtanh': torch.nn.Hardtanh(),
               'LeakyReLU': torch.nn.LeakyReLU(), 'LogSigmoid': torch.nn.LogSigmoid(),
               'PReLU': torch.nn.PReLU(), 'ReLU': torch.nn.ReLU(), 'ReLU6': torch.nn.ReLU6(),
               'RReLU': torch.nn.RReLU(), 'SELU': torch.nn.SELU(), 'CELU': torch.nn.CELU(),
               'Sigmoid': torch.nn.Sigmoid(), 'Softplus': torch.nn.Softplus(),
               'Softshrink': torch.nn.Softshrink(), 'Softsign': torch.nn.Softsign(),
               'Tanh': torch.nn.Tanh(), 'Tanhshrink': torch.nn.Tanhshrink(),
               'Hardshrink': torch.nn.Hardshrink()}

In [18]:
%%time
result = dict()

for name, activation in activations.items():
    np.random.seed(11)
    torch.manual_seed(11)
    net = SimpleNet(activation=activation)
    
    fc1_grads = []
    for x in torch.randn((NUMBER_OF_EXPERIMENTS, 1)):
        fc1_grads.append(get_fc1_grad_abs_value(net, x))
    result[name] = fc1_grads

Wall time: 1.75 s


In [19]:
mean = [(np.mean(lst), name) for name, lst in result.items()]

In [23]:
sorted(mean, reverse=True)[0]

(0.7372177013754845, 'Hardshrink')

___
___
Попробуйте решить следующую задачу пользуясь только бумагой и калькулятором:

Допустим, у нас есть нейросеть, состоящая из 4-х полносвязных слоев, в каждом из которых по одному нейрону. Для простоты будем считать, что bias'ы у нейронов отсутствуют, а все веса равны 1.

После каждого слоя мы поставим активации. В первом случае это будут tanh, во втором - ReLU. Не будем добавлять никакую loss-функцию. Тогда нашу сеть можно будет записать в виде функции:

$$ f = a_4(w_4 \cdot a_3(w_3 \cdot a_2(w_2 \cdot a_1(w_1 \cdot x)))) $$

Где $a$ - это либо tanh либо ReLU. $w_i$ - это одно число.

Пусть на вход подали x=100

Зная, что $tanh'(x) = 1 - tanh^2(x)$, рассчитайте градиенты весов сети: $[f'_{w_1}, f'_{w_2}, f'_{w_3}, f'_{w_4}, ]$ для случая a=tanh и для случая a=ReLU . Результат округлите до 3-го знака.

In [36]:
import math

t_1 = (1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*(1 - math.tanh(math.tanh(math.tanh(100)))**2)*(1 - math.tanh(math.tanh(100))**2)*(1 - math.tanh(100)**2)*100
t_2 = (1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*(1-math.tanh(math.tanh(math.tanh(100)))**2)*(1-math.tanh(math.tanh(100))**2)*math.tanh(100)
t_3 = (1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*(1-math.tanh(math.tanh(math.tanh(100)))**2)*math.tanh(math.tanh(100))
t_4 = (1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*math.tanh(math.tanh(math.tanh(100)))

[round(val, 3) for val in (t_1, t_2, t_3, t_4)]

[0.0, 0.168, 0.304, 0.436]

In [37]:
[100.0, 100.0, 100.0, 100.0]

[100.0, 100.0, 100.0, 100.0]